In [1]:
import websocket
import json
from datetime import datetime
import pubnub
import pandas as pd
from threading import Thread
from queue import Queue
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub, SubscribeCallback, PNStatusCategory

In [2]:
store = pd.HDFStore('data.h5', mode='a', complevel=5, complib='zlib', fletcher32=True)

In [3]:
coincheck_updates = Queue()
bitflyer_updates = Queue()
store_queue = Queue()

In [4]:
coincheck_orderbook = websocket.create_connection('wss://ws-api.coincheck.com/')
coincheck_orderbook.send(json.dumps({'type': "subscribe", 'channel': "btc_jpy-orderbook"}))

59

In [5]:
def collect_coincheck_updates():
        while True:
            channel, book = json.loads(coincheck_orderbook.recv())
            timestamp = datetime.now()
            coincheck_updates.put((timestamp, book))

In [6]:
def parse_coincheck_updates():
    while True:
        snapshots = [coincheck_updates.get() for _ in range(30)]
        orders = list()
        for snapshot in snapshots:
            timestamp, book = snapshot
            orders += [{'order_type': 'ask', 'price': o[0], 'volume': o[1], 'timestamp':timestamp} 
                    for o in book.get('asks')]
            orders += [{'order_type': 'bid', 'price': o[0], 'volume': o[1], 'timestamp':timestamp} 
                    for o in book.get('bids')]
        store_queue.put(('coincheck/btc_jpy', pd.DataFrame(orders)))

In [7]:
class MyListener(SubscribeCallback):
    
    def status(self, pubnub, status):
        if status.category == PNStatusCategory.PNConnectedCategory:
            print('connected')
 
    def message(self, pubnub, message):
        bitflyer_updates.put(message)
 
    def presence(self, pubnub, presence):
        print('asdad')

In [8]:
def parse_bitflyer_updates():
    while True:
        messages = [bitflyer_updates.get().message for _ in range(100)]
        store_queue.put(('bitflyet/lightning_ticker_BTC_JPY', pd.DataFrame(messages)))

In [9]:
def store_updates():
    while True:
        key, updates = store_queue.get()
        store.append(key, updates, format='table')

In [10]:
pnconfig = PNConfiguration()
pnconfig.subscribe_key = "sub-c-52a9ab50-291b-11e5-baaa-0619f8945a4f"
pnconfig.ssl = True
 
pubnub = PubNub(pnconfig)
listener = MyListener()
pubnub.add_listener(listener)
pubnub.subscribe().channels('lightning_ticker_BTC_JPY').execute()

In [11]:
Thread(target = collect_coincheck_updates).start()
Thread(target=parse_bitflyer_updates).start()
Thread(target=parse_coincheck_updates).start()
Thread(target=store_updates).start()

connected
